# Projet Arbres de décision et Méthodes d'agrégations
-- Par Isaline Hervé - M2 ECAP --

# Importation des packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer, SimpleImputer
import scipy.stats as stats

# Importation des données et premiers traitements
## Importation des données

In [2]:
df = pd.read_csv('./data/train.csv', header=0)
df.info()
# 7094 lignes, 25 colonnes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094 entries, 0 to 7093
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   nom_du_projet                        7094 non-null   object 
 1   resume_du_projet                     6917 non-null   object 
 2   montant_engage                       7094 non-null   float64
 3   numero_ej                            6025 non-null   float64
 4   numero_operateur                     1072 non-null   object 
 5   opérateur                            1072 non-null   object 
 6   demarche                             6962 non-null   object 
 7   nom_region                           6929 non-null   object 
 8   nom_departement                      6929 non-null   object 
 9   code_departement                     6929 non-null   object 
 10  siret_beneficiaire                   6930 non-null   float64
 11  raison_sociale_beneficiaire   

In [3]:
df.head()

,nom_du_projet,resume_du_projet,montant_engage,numero_ej,numero_operateur,opérateur,demarche,nom_region,nom_departement,code_departement,...,siret,activitePrincipale,codePostal,dateCreation,categorieEntreprise,trancheEffectifsUniteLegale,societeMissionUniteLegale,etatAdministratifEtablissement,anneeEffectifsEtablissement,economieSocialeSolidaireUniteLegale
0,"Changement des fenêtres ,des portes et pose de...","Suite aux différents audits, il convient de fi...",20710.00,2.104489e+09,NaN,NaN,Rénovation énergétique des bâtiments publics l...,Normandie,Eure,27,...,2.000561e+13,84.11Z,27160.0,2016-01-01,PME,12,NaN,A,2023.0,N
1,ZAC Bernard Duval,La ZAC Claude Bernard / Alexandre Duval à Renn...,200000.00,2.104469e+09,NaN,NaN,Recyclage foncier,Bretagne,Ille-et-Vilaine,35,...,5.231896e+13,42.99Z,35200.0,2010-05-10,ETI,03,NaN,A,2023.0,N
2,Travaux d'urgence de confortement et d'aménage...,Suites aux intempéries du début du mois de déc...,14157.02,2.104386e+09,NaN,NaN,Appui aux collectivités de montagne soumises à...,Provence-Alpes-Côte d'Azur,Hautes-Alpes,05,...,2.000673e+13,84.11Z,5230.0,2017-01-01,PME,12,NaN,A,2023.0,N
3,Rénovation thermique de l'école de GERBEPAL - ...,La commune de GERBÉPAL souhaite rénover le bât...,202019.00,2.104371e+09,NaN,NaN,Rénovation énergétique des bâtiments publics l...,Grand Est,Vosges,88,...,2.188020e+13,84.11Z,88430.0,1999-12-25,PME,11,NaN,A,2023.0,N
4,Rénovation énergétique du Centre Omnisport de ...,Situé à proximité du quartier de Marbé au nord...,587251.00,2.104409e+09,NaN,NaN,Rénovation énergétique des bâtiments publics l...,Bourgogne-Franche-Comté,Saône-et-Loire,71,...,2.171027e+13,84.11Z,71000.0,1983-03-01,ETI,42,NaN,A,2023.0,N


In [ ]:
# Modification des noms de colonnes
df = df.rename(columns={
    "opérateur": "operateur",
    "forme juridique_beneficiaire": "forme_juridique_beneficiaire",
    "activitePrincipale": "activite_principale",
    "codePostal": "code_postal",
    "dateCreation": "date_creation",
    "categorieEntreprise": "categorie_entreprise",
    "trancheEffectifsUniteLegale": "tranche_effectifs_unite_legale",
    "societeMissionUniteLegale": "societe_mission_unite_legale",
    "etatAdministratifEtablissement": "etat_administratif_etablissement",
    "anneeEffectifsEtablissement": "annee_effectifs_etablissement",
    "economieSocialeSolidaireUniteLegale": "economie_sociale_solidaire_unite_legale"
})

# Vérification 
df.head()

,nom_du_projet,resume_du_projet,montant_engage,numero_ej,numero_operateur,operateur,demarche,nom_region,nom_departement,code_departement,...,siret,activitePrincipale,codePostal,dateCreation,categorieEntreprise,trancheEffectifsUniteLegale,societeMissionUniteLegale,etatAdministratifEtablissement,anneeEffectifsEtablissement,economieSocialeSolidaireUniteLegale
0,"Changement des fenêtres ,des portes et pose de...","Suite aux différents audits, il convient de fi...",20710.00,2.104489e+09,NaN,NaN,Rénovation énergétique des bâtiments publics l...,Normandie,Eure,27,...,2.000561e+13,84.11Z,27160.0,2016-01-01,PME,12,NaN,A,2023.0,N
1,ZAC Bernard Duval,La ZAC Claude Bernard / Alexandre Duval à Renn...,200000.00,2.104469e+09,NaN,NaN,Recyclage foncier,Bretagne,Ille-et-Vilaine,35,...,5.231896e+13,42.99Z,35200.0,2010-05-10,ETI,03,NaN,A,2023.0,N
2,Travaux d'urgence de confortement et d'aménage...,Suites aux intempéries du début du mois de déc...,14157.02,2.104386e+09,NaN,NaN,Appui aux collectivités de montagne soumises à...,Provence-Alpes-Côte d'Azur,Hautes-Alpes,05,...,2.000673e+13,84.11Z,5230.0,2017-01-01,PME,12,NaN,A,2023.0,N
3,Rénovation thermique de l'école de GERBEPAL - ...,La commune de GERBÉPAL souhaite rénover le bât...,202019.00,2.104371e+09,NaN,NaN,Rénovation énergétique des bâtiments publics l...,Grand Est,Vosges,88,...,2.188020e+13,84.11Z,88430.0,1999-12-25,PME,11,NaN,A,2023.0,N
4,Rénovation énergétique du Centre Omnisport de ...,Situé à proximité du quartier de Marbé au nord...,587251.00,2.104409e+09,NaN,NaN,Rénovation énergétique des bâtiments publics l...,Bourgogne-Franche-Comté,Saône-et-Loire,71,...,2.171027e+13,84.11Z,71000.0,1983-03-01,ETI,42,NaN,A,2023.0,N


## Vérification et traitement des doublons

In [5]:
# Vérification des doublons 
dupes = df.duplicated().sum()
print(f"Number of duplicate rows: {dupes}")
# Pas de doublons détectés

Number of duplicate rows: 0


## Vérification et traitement des valeurs manquantes

In [6]:
# Vérification des valeurs manquantes
print(f"Number of missing values per column:\n{df.isnull().sum().sort_values(ascending=False)}")
# Beaucoup de valeurs manquantes, notamment dans societeMissionUniteLegale, operateur et numero_operateur
# Pas de valeurs manquantes dans nom_du_projet et montant_engage

Number of missing values per column:
societeMissionUniteLegale              7082
operateur                              6022
numero_operateur                       6022
economieSocialeSolidaireUniteLegale    1966
numero_ej                              1069
anneeEffectifsEtablissement             394
etatAdministratifEtablissement          237
categorieEntreprise                     218
resume_du_projet                        177
codePostal                              168
trancheEffectifsUniteLegale             165
dateCreation                            165
nom_region                              165
nom_departement                         165
siret                                   165
code_departement                        165
activitePrincipale                      165
siret_beneficiaire                      164
raison_sociale_beneficiaire             138
forme_juridique_beneficiaire            138
demarche                                132
code_commune                           

In [7]:
# Visualisation des valeurs manquantes en pourcentage
na = df.isna().sum()/df.shape[0]*100 # pourcentage de valeurs manquantes par variable
palettehex = sns.color_palette("Paired").as_hex()
px.bar(na.sort_values(),
         title = "Pourcentage de valeurs manquantes par variable",
         color_discrete_sequence = palettehex[0:1],
         template='ggplot2',
         range_y = [0,100],
         width=800,
         labels={'value':'%', 'index':''},
         color=None
        ).update_traces(showlegend=False).update_xaxes(tickangle=45)
# societeMissionUniteLegale, numero_operateur et operateur ont plus de 70% de valeurs manquantes

In [ ]:
# Traitement des valeurs manquantes
NON_RENSEIGNE = "Non_renseigne"

# Colonnes où toute valeur manquante entraîne la suppression de la ligne (car pas de sens à imputer)
DROP_ROW_IF_NA = [
    # Identifiants
    "nom_du_projet",
    "numero_ej",
    "numero_operateur",
    "operateur",
    # Catégories d'entreprise
    "categorie_entreprise",
    "categorie_juridique_unite_legale",
    "tranche_effectifs_unite_legale",
    # Temporel
    "date_creation",
    # Activité
    "activite_principale",
    # Administratif
    "etat_administratif_etablissement",
    "annee_effectifs_etablissement"
]

# Colonnes où les valeurs manquantes doivent être remplacées
REPLACE_NA_COLS = [
    # Géographie à l'échelle communale
    "code_commune",
    "nom_commune",
    # Binaires institutionnelles
    "societe_mission_unite_legale",
    "economie_sociale_solidaire_unite_legale"
]

df = df.copy()

# 1 - Suppression des colonnes avec > 30 % de valeurs manquantes
missing_rate_cols = df.isna().mean()
cols_to_drop = missing_rate_cols[missing_rate_cols > 0.30].index.tolist()
df.drop(columns=cols_to_drop, inplace=True)

# 2 - > 5 % de manquants → "Non_renseigne"
# (appliquée après suppression des colonnes > 30 %)
missing_rate_cols = df.isna().mean()
cols_to_fill = missing_rate_cols[
    (missing_rate_cols > 0.05) & (missing_rate_cols <= 0.30)
].index.tolist()

for col in cols_to_fill:
    if col in df.columns and df[col].dtype == "object":
        df[col] = df[col].fillna(NON_RENSEIGNE)

# 3 - Remplacement explicite par "Non_renseigne"
for col in REPLACE_NA_COLS:
    if col in df.columns:
        df[col] = df[col].fillna(NON_RENSEIGNE)

# 4 - Suppression des lignes avec valeurs manquantes
existing_drop_cols = [c for c in DROP_ROW_IF_NA if c in df.columns]
df.dropna(subset=existing_drop_cols, inplace=True)

# Vérification finale
print("Shape finale :", df.shape)
print("\nColonnes restantes :", len(df.columns))
print("\nTop 10 taux de valeurs manquantes restants :")
print(df.isna().mean().sort_values(ascending=False).head(10))

Shape finale : (6025, 22)

Colonnes restantes : 22

Top 10 taux de valeurs manquantes restants :
anneeEffectifsEtablissement       0.040332
etatAdministratifEtablissement    0.018921
categorieEntreprise               0.016929
resume_du_projet                  0.010622
codePostal                        0.009793
nom_region                        0.009295
trancheEffectifsUniteLegale       0.009295
activitePrincipale                0.009295
siret                             0.009295
dateCreation                      0.009295
dtype: float64


In [ ]:
# Traitement des valeurs manquantes

# economieSocialeSolidaireUniteLegale
# 

class MissingValuesProcessor:
    """
    Traitement des valeurs manquantes dans un DataFrame pandas.

    - Supprime les colonnes avec plus de `missing_threshold` de valeurs manquantes
    - Impute :
        * montant_engage (float quantitatif) : IterativeImputer
        * autres float (identifiants déguisés) : catégorie 'Manquant'
        * catégorielles : catégorie 'Manquant'
    """

    def __init__(
        self,
        missing_threshold: float = 0.30,
        montant_col: str = "montant_engage",
        random_state: int = 0,
        max_iter: int = 10,
    ):
        self.missing_threshold = missing_threshold
        self.montant_col = montant_col
        self.random_state = random_state
        self.max_iter = max_iter

        self.dropped_columns_ = None
        self.montant_present_ = False
        self.other_float_cols_ = None
        self.cat_cols_ = None
        self.imputer_montant_ = None

    def fit(self, df: pd.DataFrame):
        # 1) Suppression colonnes trop manquantes
        missing_ratio = df.isna().mean()
        self.dropped_columns_ = (
            missing_ratio[missing_ratio > self.missing_threshold]
            .index
            .tolist()
        )

        df_reduced = df.drop(columns=self.dropped_columns_)

        # 2) Détection de montant_engage
        self.montant_present_ = (
            self.montant_col in df_reduced.columns
            and df_reduced[self.montant_col].dtype == "float64"
        )

        # 3) Autres floats = identifiants → catégorielles
        float_cols = df_reduced.select_dtypes(include=["float64"]).columns.tolist()
        self.other_float_cols_ = [
            c for c in float_cols if c != self.montant_col
        ]

        # 4) Colonnes catégorielles (object + category)
        self.cat_cols_ = (
            df_reduced
            .select_dtypes(include=["object", "category"])
            .columns
            .tolist()
        )

        # 5) Imputer montant_engage uniquement
        if self.montant_present_:
            self.imputer_montant_ = IterativeImputer(
                max_iter=self.max_iter,
                random_state=self.random_state,
            )
            self.imputer_montant_.fit(
                df_reduced[[self.montant_col]]
            )

        return self

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        df_reduced = df.drop(columns=self.dropped_columns_).copy()

        # 1) Imputation montant_engage
        if self.montant_present_:
            df_reduced[self.montant_col] = (
                self.imputer_montant_
                .transform(df_reduced[[self.montant_col]])
                .ravel()
            )

        # 2) Autres floats → string + 'Manquant'
        for col in self.other_float_cols_:
            df_reduced[col] = (
                df_reduced[col]
                .astype("string")
                .fillna("Manquant")
            )

        # 3) Catégorielles → ajout explicite 'Manquant'
        for col in self.cat_cols_:
            df_reduced[col] = (
                df_reduced[col]
                .astype("string")
                .fillna("Manquant")
                .astype("category")
            )

        return df_reduced

    def fit_transform(self, df: pd.DataFrame) -> pd.DataFrame:
        return self.fit(df).transform(df)


In [9]:
processor = MissingValuesProcessor(missing_threshold=0.30)
df_imputed = processor.fit_transform(df)

print("Colonnes supprimées :", processor.dropped_columns_)
print("Colonnes numériques :", processor.num_cols_)
print("Colonnes catégorielles :", processor.cat_cols_)

# Vérification finale
na_ratio = df_imputed.isna().mean().sort_values(ascending=False)
print(na_ratio)

Colonnes supprimées : ['numero_operateur', 'operateur', 'societeMissionUniteLegale']
Colonnes numériques : ['montant_engage', 'numero_ej', 'siret_beneficiaire', 'siret', 'codePostal', 'anneeEffectifsEtablissement']
Colonnes catégorielles : ['nom_du_projet', 'resume_du_projet', 'demarche', 'nom_region', 'nom_departement', 'code_departement', 'raison_sociale_beneficiaire', 'forme_juridique_beneficiaire', 'code_commune', 'nom_commune', 'activitePrincipale', 'dateCreation', 'categorieEntreprise', 'trancheEffectifsUniteLegale', 'etatAdministratifEtablissement', 'economieSocialeSolidaireUniteLegale']
nom_du_projet                          0.0
resume_du_projet                       0.0
montant_engage                         0.0
numero_ej                              0.0
demarche                               0.0
nom_region                             0.0
nom_departement                        0.0
code_departement                       0.0
siret_beneficiaire                     0.0
raison_soci

In [10]:
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094 entries, 0 to 7093
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   nom_du_projet                        7094 non-null   object 
 1   resume_du_projet                     7094 non-null   object 
 2   montant_engage                       7094 non-null   float64
 3   numero_ej                            7094 non-null   float64
 4   demarche                             7094 non-null   object 
 5   nom_region                           7094 non-null   object 
 6   nom_departement                      7094 non-null   object 
 7   code_departement                     7094 non-null   object 
 8   siret_beneficiaire                   7094 non-null   float64
 9   raison_sociale_beneficiaire          7094 non-null   object 
 10  forme_juridique_beneficiaire         7094 non-null   object 
 11  code_commune                  

In [ ]:
# Conversion des colonnes numériques en string
cols_numeric_id = [
    "numero_ej",
    "siret_beneficiaire",
    "siret",
    "codePostal",
]

for col in cols_numeric_id:
    df_imputed[col] = df_imputed[col].astype("Int64").astype("string")

# Conversion des colonnes alphanumériques en string
cols_alpha_id = [
    "code_commune",
    "code_departement",
    # "numero_operateur" (a été supprimée)
]

for col in cols_alpha_id:
    df_imputed[col] = df_imputed[col].astype("string")

# Conversion des colonnes avec nombre entier en Int64
cols_to_int = [
    "anneeEffectifsEtablissement"
]

for col in cols_to_int:
    df_imputed[col] = df_imputed[col].astype("Int64") # Int64 pour la gestion des NaN

# Conversion des colonnes objects en catégorielles
categorical_cols = [
    # "operateur", (a été supprimée)
    "nom_region",
    "nom_departement",
    "code_departement",
    "forme_juridique_beneficiaire",
    "activitePrincipale",
    "categorieEntreprise",
    "trancheEffectifsUniteLegale",
    # "societeMissionUniteLegale", (a été supprimée)
    "etatAdministratifEtablissement",
    "economieSocialeSolidaireUniteLegale",
]

df_imputed[categorical_cols] = df_imputed[categorical_cols].astype("category")

# Conversion des variables date en format datetime
df_imputed["dateCreation"] = pd.to_datetime(df_imputed["dateCreation"], errors="coerce")

# Variables textes
text_cols = [
    "nom_du_projet",
    "resume_du_projet",
    "demarche",
    "raison_sociale_beneficiaire",
]

df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094 entries, 0 to 7093
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   nom_du_projet                        7094 non-null   object        
 1   resume_du_projet                     7094 non-null   object        
 2   montant_engage                       7094 non-null   float64       
 3   numero_ej                            7094 non-null   string        
 4   demarche                             7094 non-null   object        
 5   nom_region                           7094 non-null   category      
 6   nom_departement                      7094 non-null   category      
 7   code_departement                     7094 non-null   category      
 8   siret_beneficiaire                   7094 non-null   string        
 9   raison_sociale_beneficiaire          7094 non-null   object        
 10  forme_juridi

## Création de nouvelles variables


# Statistiques descriptives
## Statistiques descriptives univariées

In [ ]:
# Distribution

In [ ]:
# Boxplot - Valeurs extrêmes

## Statistiques descriptives bivariées

In [ ]:
# Corrélation entre variables explicatives

In [ ]:
# Corrélation entre variables explicatives et variable cible

# Encodage des variables catégorielles

# Modélisation